In [5]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences
from time import sleep
from tqdm import tqdm  # optional: pip install tqdm


SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")

SYSTEM_PROMPT_IMAGINE = """
Your task is to take a human imaginative directive and translate it into a coherent, structured list of domains, technologies, and disciplines that could be explored to realize or approximate that imaginative directive.
The goal is to break down the directive into actionable or researchable avenues across science, technology, psychology, education, or other relevant fields.
Each entry should be self-contained, meaning it should be understandable on its own, and clearly tied back to the core imaginative directive.
"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}



def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def sample_text(text, x):
    max_start = max(0, len(text) - x)
    random_start = random.randint(0, max_start)
    excerpt = text[random_start:random_start + x]
    return excerpt


load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

DB_FILE = "library_temp.sqlite"
MAX_RESULTS = 100



In [2]:
imagination = {}

In [16]:
SYSTEM_PROMPT_SEED_FANTASIA = """
You are a Fantasiagenesis Architect.

Your task is to analyze a given excerpt and determine whether it can be used to create or support the given fantasia (a wish, dream, or imaginative goal).

Follow these steps:

1. Read the excerpt carefully and identify any concrete principles, processes, systems, or structures it describes.
2. Assess whether these elements can be **clearly and meaningfully** applied—directly or by strong analogy—to the realization of the given fantasia.
3. If a meaningful connection can be made, explain how the information could be used to create or move toward the fantasia. Be grounded: use only what is present in the excerpt or is a reasonable extension from it.
4. If no clear connection can be made, **return only**: `Nothing`.

Your response must follow this structure:
- **Excerpt Summary**: A 1–2 sentence summary of the main ideas in the excerpt.
- **Fantasia Mapping**: If possible, show how the excerpt’s principles can be used toward achieving the fantasia.

"""

USER_PROMPT_SEED_FANTASIA = """
Excerpt:
'''
{excerpt}
'''

Fantasia: {fantasia}

How can the information in this excerpt be used to create the given fantasia?

"""

In [3]:
fantasia = "I imagine instantly learning any topic."

sys_msg = SYSTEM_PROMPT_SEED_FANTASIA
usr_msg = USER_PROMPT_SEED_FANTASIA.format(excerpt=excerpt, fantasia=fantasia)

resp = ask(sys_msg, usr_msg)
print(seeds_resp)


To realize or approximate the imaginative directive of "instantly learning any topic," various domains, technologies, and disciplines can be explored. Here is a structured list to guide research and development efforts:

1. **Neuroscience and Cognitive Science:**
   - **Neural Plasticity:** Study brain plasticity to understand how the brain adapts to new information.
   - **Memory Encoding and Retrieval:** Explore mechanisms of how memories are encoded, stored, and retrieved.
   - **Neurogenesis and Synaptic Formation:** Investigate how new neurons and synaptic connections are formed during learning.

2. **Brain-Machine Interfaces:**
   - **Direct Information Transmission:** Research interfaces that could transmit data directly into the brain.
   - **Neurofeedback Systems:** Develop systems that provide real-time feedback to optimize learning processes.
   - **Wireless Brain Connectivity:** Explore technologies for wireless communication between neural activity and external devices.

3

NameError: name 'ask_func' is not defined

In [12]:
KEYWORDS = ["diamond"]
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
papers = {}
for r in search_results:
    title, text = r['title'], r['text']
    papers[title] = text
    print(title, len(text))


Diamond Handbook: How To Look At Diamonds & Avoid Ripoffs 319838
The Diamond Book: CIBJO Diamond Commission 2022-1 59207
Diamond: Electronic Properties and Applications 1154803
Diamonds: An Early History of the King of Gems 693764
Legend of the Rock: A Two Thousand Year History of the Diamond 21569
Diamonds 142168
Lifespan HCP 2.0 Data Release Appendix 2: Details and References for Behavioral & Clinical Instruments 169592
Neurogenesis and Neural Plasticity 1269182
None 6248598
Principles of Neural Science, Fifth Edition 6248598


In [17]:
for n in range(len(papers)):
    title, text = list(papers.keys())[n], list(papers.values())[n]
    excerpt = sample_text(text, 10000)
    fantasia = "I imagine instantly learning any topic."

    sys_msg = SYSTEM_PROMPT_SEED_FANTASIA
    usr_msg = USER_PROMPT_SEED_FANTASIA.format(excerpt=excerpt, fantasia=fantasia)


    resp = ask(sys_msg, usr_msg)
    print(resp)

- **Excerpt Summary**: The excerpt provides a detailed history and description of jewelry styles and techniques from the Georgian to the Edwardian period, highlighting changes in materials, design motifs, and cutting techniques influenced by discoveries and technological advancements.

- **Fantasia Mapping**: The excerpt describes how advancements in technology and material availability led to significant shifts in jewelry design and production, such as the development of new gem-cutting techniques or the introduction of platinum in jewelry construction. This historical analysis can inspire the fantasia by highlighting the role of innovation and systematic learning in evolving an art form. To achieve the fantasia of instantly learning any topic, one could draw an analogy to the process by which jewelers gradually refined their skills and knowledge through systematic advancements over time. Imagine a scenario where a learning system leverages historical and practical data, much like the